In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from A.srcnn_data import preprocess_image, srcnn_image_generator, display_comparison
from A.srcnn import create_srcnn_model, psnr, predict_images, compare_average_psnr

from B.espcn_data import espcn_image_generator
from B.espcn import create_espcn_model

In [ ]:
# List all the physical GPUs available
gpus = tf.config.experimental.list_physical_devices('GPU')

# This allows TensorFlow to automatically use as much GPU memory as needed and only when required.
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Preprocess data

In [ ]:
# Define the directories where the training and validation images are stored
train_hr_dir = 'Datasets/DIV2K_train_HR' 
train_lr_bicubic_dir = 'Datasets/DIV2K_train_LR_bicubic_X4'
train_lr_unknown_dir = 'Datasets/DIV2K_train_LR_unknown_X4'
valid_hr_dir = 'Datasets/DIV2K_valid_HR'
valid_lr_bicubic_dir = 'Datasets/DIV2K_valid_LR_bicubic_X4'
valid_lr_unknown_dir = 'Datasets/DIV2K_valid_LR_unknown_X4'

# Preprocess the validation images
# This involves loading the images, cropping them to the specified size, and normalizing their pixel values
valid_hr_ds = preprocess_image(valid_hr_dir, 600)
valid_lr_bicubic_ds = preprocess_image(valid_lr_bicubic_dir, 150)
valid_lr_unknown_ds = preprocess_image(valid_lr_unknown_dir, 150)

# Resize the low resolution validation images to the same size as the high resolution images
# This is done using bicubic interpolation, which is a common method for image resizing
valid_lr_bicubic_ds_resized = np.array([cv2.resize(img, (600, 600), interpolation=cv2.INTER_CUBIC) for img in valid_lr_bicubic_ds])
valid_lr_unknown_ds_resized = np.array([cv2.resize(img, (600, 600), interpolation=cv2.INTER_CUBIC) for img in valid_lr_unknown_ds])

In [ ]:
# Define the target size and batch size for the image generators
low_res_target_size = 150
high_res_target_size = 600
generator_batch_size = 2

# These generators will load batches of images from the specified directories, resize them to the target sizes, and yield them for training the model
srcnn_bicubic_gen = srcnn_image_generator(train_lr_bicubic_dir, train_hr_dir, low_res_target_size, high_res_target_size, generator_batch_size)
srcnn_unknown_gen = srcnn_image_generator(train_lr_unknown_dir, train_hr_dir, low_res_target_size, high_res_target_size, generator_batch_size)
espcn_bicubic_gen = espcn_image_generator(train_lr_bicubic_dir, train_hr_dir, low_res_target_size, high_res_target_size, generator_batch_size)
espcn_unknown_gen = espcn_image_generator(train_lr_unknown_dir, train_hr_dir, low_res_target_size, high_res_target_size, generator_batch_size)


Track 1: Bicubic x4 SRCNN

In [ ]:
# Train the SRCNN model for the bicubic downsampled images 7,5hrs
# bicubic_srcnn_model = create_srcnn_model()
# bicubic_srcnn_model.summary()
# bicubic_srcnn_model.fit(bicubic_gen, steps_per_epoch=2000, epochs=50)
# bicubic_srcnn_model.save('A/bicubic_srcnn_model.h5')

In [ ]:
bicubic_srcnn_model = load_model('A/bicubic_srcnn_model.h5', custom_objects={'psnr': psnr})

prediction_batch_size = 10  # Set your batch size
srcnn_predicted_bicubic_hr_images = predict_images(bicubic_srcnn_model, valid_lr_bicubic_ds_resized, prediction_batch_size)

bicubic_original_psnr, bicubic_srcnn_average_psnr = compare_average_psnr(valid_hr_ds, srcnn_predicted_bicubic_hr_images, valid_lr_bicubic_ds_resized)

print(f'Original_Average PSNR: {bicubic_original_psnr}dB')
print(f'SRCNN_Average PSNR: {bicubic_srcnn_average_psnr}dB')

In [ ]:
indices = np.random.choice(len(srcnn_predicted_bicubic_hr_images), 3, replace=False)

In [ ]:
# Select the images at these indices
selected_predicted_images = srcnn_predicted_bicubic_hr_images[indices]
selected_true_hr_images = valid_hr_ds[indices]
selected_low_res_images = valid_lr_bicubic_ds_resized[indices]

# Now you can display and compare the selected images
display_comparison(selected_low_res_images, selected_predicted_images, selected_true_hr_images, 'bicubic_srcnn_random_comparison')

Track 1: Bicubic x4 ESPCN

In [ ]:
bicubic_espcn_model = create_espcn_model()
bicubic_espcn_model.summary()
bicubic_espcn_model.fit(espcn_bicubic_gen, steps_per_epoch=20, epochs=50)
bicubic_espcn_model.save('A/bicubic_espcn_model.h5')

In [ ]:
bicubic_espcn_model = load_model('A/bicubic_espcn_model.h5', custom_objects={'psnr': psnr})
espcn_predicted_bicubic_hr_images = predict_images(bicubic_espcn_model, valid_lr_bicubic_ds, prediction_batch_size)

bicubic_original_psnr, bicubic_espcn_average_psnr = compare_average_psnr(valid_hr_ds, espcn_predicted_bicubic_hr_images, valid_lr_bicubic_ds_resized)

print(f'Original_Average PSNR: {bicubic_original_psnr}dB')
print(f'SRCNN_Average PSNR: {bicubic_espcn_average_psnr}dB')

In [ ]:
# Select the images at these indices
selected_predicted_images = espcn_predicted_bicubic_hr_images[indices]
selected_true_hr_images = valid_hr_ds[indices]
selected_low_res_images = valid_lr_bicubic_ds_resized[indices]

# Now you can display and compare the selected images
display_comparison(selected_low_res_images, selected_predicted_images, selected_true_hr_images, 'bicubic_espcn_random_comparison')

Track 2: Unknown x4 SRCNN

In [ ]:
# Train the SRCNN model for the unknown downsampled images 7,5hrs
# unknown_srcnn_model = create_srcnn_model()
# unknown_srcnn_model.summary()
# unknown_srcnn_model.fit(srcnn_unknown_gen, steps_per_epoch=2000, epochs=50)
# unknown_srcnn_model.save('B/unknown_srcnn_model.h5')

In [ ]:
unknown_srcnn_model = load_model('B/unknown_srcnn_model.h5', custom_objects={'psnr': psnr})
srcnn_predicted_unknown_hr_images = predict_images(unknown_srcnn_model, valid_lr_unknown_ds_resized, prediction_batch_size)

unknown_original_psnr, unknown_srcnn_average_psnr = compare_average_psnr(valid_hr_ds, srcnn_predicted_unknown_hr_images, valid_lr_unknown_ds_resized)

print(f'Original_Average PSNR: {unknown_original_psnr}dB')
print(f'SRCNN_Average PSNR: {unknown_srcnn_average_psnr}dB')

In [ ]:
# Select the images at these indices
selected_predicted_images = srcnn_predicted_unknown_hr_images[indices]
selected_true_hr_images = valid_hr_ds[indices]
selected_low_res_images = valid_lr_unknown_ds_resized[indices]

# Now you can display and compare the selected images
display_comparison(selected_low_res_images, selected_predicted_images, selected_true_hr_images, 'unknown_srcnn_random_comparison')

Track 2: Unknown x4 ESPCN

In [ ]:
unknown_espcn_model = create_espcn_model()
unknown_espcn_model.summary()
unknown_espcn_model.fit(espcn_unknown_gen, steps_per_epoch=20, epochs=50)
unknown_espcn_model.save('B/unknown_espcn_model.h5')

In [ ]:
unknown_espcn_model = load_model('B/unknown_espcn_model.h5', custom_objects={'psnr': psnr})
espcn_predicted_unknown_hr_images = predict_images(unknown_espcn_model, valid_lr_unknown_ds, prediction_batch_size)

unknown_original_psnr, unknown_espcn_average_psnr = compare_average_psnr(valid_hr_ds, espcn_predicted_unknown_hr_images, valid_lr_unknown_ds_resized)

print(f'Original_Average PSNR: {unknown_original_psnr}dB')
print(f'SRCNN_Average PSNR: {unknown_espcn_average_psnr}dB')

In [ ]:
# Select the images at these indices
selected_predicted_images = espcn_predicted_unknown_hr_images[indices]
selected_true_hr_images = valid_hr_ds[indices]
selected_low_res_images = valid_lr_unknown_ds_resized[indices]

# Now you can display and compare the selected images
display_comparison(selected_low_res_images, selected_predicted_images, selected_true_hr_images, 'unknown_espcn_random_comparison')